In [1]:
!npm install -g localtunnel
!pip install flask-cors

/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
added 22 packages from 22 contributors in 3.799s


In [2]:

import zipfile
import os
from flask import Flask, request
from flask_cors import CORS

app = Flask(__name__)
CORS(app)  # Permitir solicitudes CORS
UPLOAD_FOLDER = '/content/uploads'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

@app.route('/upload', methods=['POST'])
def upload_file():
    if 'file' not in request.files:
        return 'No file part', 400
    file = request.files['file']
    if file.filename == '':
        return 'No selected file', 400
    if file:
        filename = file.filename
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(file_path)
        return 'File uploaded successfully', 200


In [3]:

def process_uploaded_file(zip_path):
    extract_dir = '/content/extracted'
    os.makedirs(extract_dir, exist_ok=True)

    # Extraer archivo zip
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)

    # Listar archivos extraídos
    extracted_files = os.listdir(extract_dir)
    print(extracted_files)

    # Aquí puedes agregar el código para procesar los archivos extraídos
    # ...

# Comprobar si hay archivos en el directorio de subidas y procesarlos
uploaded_files = os.listdir(UPLOAD_FOLDER)
for file in uploaded_files:
    if file.endswith('.zip'):
        process_uploaded_file(os.path.join(UPLOAD_FOLDER, file))


In [4]:
from flask import Flask, request, jsonify, send_from_directory
from flask_cors import CORS
import json
import threading

@app.route('/web/<path:path>')
def serve_file(path):
    return send_from_directory('/content/web', path)

@app.route('/modify', methods=['POST'])
def modify_config():
    new_value = request.json.get('ntxCafCompressed')
    with open('/content/web/config.json', 'r') as config_file:
        config = json.load(config_file)

    config['ntxCafCompressed'] = new_value

    with open('/content/web/config.json', 'w') as config_file:
        json.dump(config, config_file)

    return jsonify({"status": "success"})

def run_app():
    app.run(host='0.0.0.0', port=5000)

thread = threading.Thread(target=run_app)
thread.start()


 * Serving Flask app '__main__'
 * Debug mode: off


In [5]:
import time
time.sleep(5)  # Espera a que localtunnel se inicie

# Obtener la contraseña del túnel
tunnel_password = os.popen('curl https://loca.lt/mytunnelpassword').read().strip()
print(f"Tunnel password: {tunnel_password}")

INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


Tunnel password: 34.30.225.119


In [ ]:
!lt --port 5000

your url is: https://wide-trees-cry.loca.lt
